In [1]:
import pandas as pd
import nltk
import hashlib

In [2]:
def kgrams_hash(sentence, k=5):
    hashes = []
    words = nltk.word_tokenize(sentence)
    for i in range(0, len(words) - k + 1):
        mystring = ' '.join(words[i:i+k])
        hash_object = hashlib.md5(mystring.encode())
        hsh = hash_object.hexdigest()
        hashes.append(hsh)
    return hashes
        

In [24]:
def winnow(text, k=5, w=6):
    fingerprints = set([])
    sentences = nltk.sent_tokenize(text)
    dic = {}
    for index, sentence in enumerate(sentences):
        #if sentence
        sentence_hashes = kgrams_hash(sentence, k)
#         print (sentence_hashes)
        if len(sentence_hashes) and len(sentence_hashes) < w:
#             print ("small")
            fingerprints.add(min(sentence_hashes))
#             dic[min(sentence_hashes)] = index
        else:
            for i in range(0, len(sentence_hashes) - w + 1):
                window = sentence_hashes[i:i+w]
                fingerprints.add(min(window))
#                 dic[min(window)] = index
#                 print (window)
#     return sentences, dic, fingerprints
    return fingerprints

In [15]:
def compare(text1, text2):
    st1, dic1, fp1 = winnow(text1)
    st2, dic2, fp2 = winnow(text2)
    common = fp1 & fp2
#     for h in common:
#         print ("Common Sentences: ")
#         print ("First text: ", st1[dic1[h]])
#         print ("Second text: ", st2[dic2[h]])
#     print ("Fingerprints in text1: ", len(fp1))
#     print ("Fingerprints in text2: ", len(fp2))
#     print ("Common Fingerprints: ", len(common))
    return len(common) / min((len(fp1),len(fp2)))

In [7]:
df = pd.read_json('data/Cell_Phones_and_Accessories_5.json', lines=True)

In [9]:
reviews = df['reviewText']

In [16]:
compare(reviews[0], reviews[1])

0.0

In [19]:
for index, review in enumerate(reviews):
    score = compare(reviews[0], reviews[1])
    if score:
        print (score)
    if (index % 1000 == 0):
        print (index)

0
1000
2000
3000
4000
5000
6000
7000
8000
9000
10000
11000
12000
13000
14000
15000
16000
17000
18000
19000
20000
21000
22000
23000
24000
25000
26000
27000
28000
29000
30000
31000
32000
33000
34000
35000
36000
37000
38000
39000
40000
41000
42000
43000
44000
45000
46000
47000
48000
49000
50000
51000
52000
53000
54000
55000
56000
57000
58000
59000
60000
61000
62000
63000
64000
65000
66000
67000
68000
69000
70000
71000
72000
73000
74000
75000
76000
77000
78000
79000
80000
81000
82000
83000
84000
85000
86000
87000
88000
89000
90000
91000
92000
93000
94000
95000
96000
97000
98000
99000
100000
101000
102000
103000
104000
105000
106000
107000
108000
109000
110000
111000
112000
113000
114000
115000
116000
117000
118000
119000
120000
121000
122000
123000
124000
125000
126000
127000
128000
129000
130000
131000
132000
133000
134000
135000
136000
137000
138000
139000
140000
141000
142000
143000
144000
145000
146000
147000
148000
149000
150000
151000
152000
153000
154000
155000
156000
157000
158000


In [49]:
num_reviews = 10000

In [56]:
reviews.size

194439

In [51]:
fingerprints = []
curr_review = reviews[0:num_reviews]
for index, rev in enumerate(curr_review):
    if (index % 1000 == 0):
        print ("Fingerprints Computed: ", index)
    fingerprints.append(winnow(rev))

Fingerprints Computed:  0
Fingerprints Computed:  1000
Fingerprints Computed:  2000
Fingerprints Computed:  3000
Fingerprints Computed:  4000
Fingerprints Computed:  5000
Fingerprints Computed:  6000
Fingerprints Computed:  7000
Fingerprints Computed:  8000
Fingerprints Computed:  9000


In [116]:
plg_indices=[]
for i in range(num_reviews):
    for j in range(i +1, num_reviews):
        common = fingerprints[i] & fingerprints[j]
        score = 0
        if len(fingerprints[i]) and len(fingerprints[j]):
            score = len(common)/min(len(fingerprints[i]), len(fingerprints[j]))
        if score > 0.5:
            plg_indices.append((i, j, score))
#             print (score)
#             print (i, j)
    if (i % 1000 == 0):
        print ("Completed: ", i)

Completed:  0
Completed:  1000
Completed:  2000
Completed:  3000
Completed:  4000
Completed:  5000
Completed:  6000
Completed:  7000
Completed:  8000
Completed:  9000


In [118]:
len(plg_indices)

65

In [46]:
reviews[715]

'I am not impressed.I don\'t know what the other reviewers are smoking. This device does not cancel out background noise. Listeners on the other end say I sound like I\'m in a tunnel. Two strikes.As you can see from the pictures there is quite a distance between one\'s mouth and the "pinhole" for the internal microphone. There is no way to cancel out background noise lest your own voice be considered to be background noise. This is a defect in all devices that have a similar design.The earpiece is not suited for driving or walking. The "ear attachment" and its hinge is made of flimsy plastic. As a result the fit isn\'t tight enough. I often need to turn my head quickly while driving and it feels like the thing\'s going to fly off my head into the back seat. It\'s an extremely annoying distraction that you don\'t want to have while driving.On the good side, I don\'t have any problems hearing and the volume controls are very convenient to use.'

In [47]:
reviews[735]

'I am not impressed.I don\'t know what the other reviewers are smoking.  This device does not cancel out background noise.  Listeners on the other end say I sound like I\'m in a tunnel.  Two strikes.As you can see from the pictures there is quite a distance between one\'s mouth and the "pinhole" for the internal microphone.  There is no way to cancel out background noise lest your own voice be considered to be background noise.  This is a design defect in all devices that have a similar design.The earpiece is not suited for driving or walking.  The "ear attachment" and its hinge is made of flimsy loose plastic.  As a result the fit isn\'t tight enough.  I often need to turn my head quickly while driving and it feels like the thing\'s going to fly off my head into the back seat.  It\'s an extremely annoying distraction that you don\'t want to have while driving.On the good side, I don\'t have any problems hearing and the volume controls are very convenient to use.'

In [119]:
num_reviews = 10000
plg_indices = []
plg_dic = {}
for i in range(num_reviews):
    for fp in fingerprints[i]:
        if fp in plg_dic:
            indices = plg_dic[fp]
            indices.append(i)
            plg_dic[fp] = indices
        else:
            plg_dic[fp] = [i]

In [120]:
plg_indices = []
for items in plg_dic:
    if (len(plg_dic[items]) > 1):
        temp = plg_dic[items]
        sz = len(temp)
        for i in range(sz):
            for j in range(i + 1, sz):
                plg_indices.append((temp[i], temp[j]))
            

In [121]:
plg_indices = set(plg_indices)

In [131]:
scores = []
sentence_threshold = 2 #minimum number of windows
for items in plg_indices:
    i = items[0]
    j = items[1]
    common = fingerprints[i] & fingerprints[j]
    temp = min(len(fingerprints[i]), len(fingerprints[j]))
    score = len(common)/temp
    if score > 0.5 and temp >= sentence_threshold:
        scores.append((i, j, score))

In [132]:
len(scores)

55

In [133]:
scores

[(4140, 6738, 0.9333333333333333),
 (7030, 7258, 1.0),
 (6038, 9887, 0.5384615384615384),
 (2728, 2747, 0.75),
 (2776, 7169, 0.6666666666666666),
 (1658, 2061, 1.0),
 (8562, 8600, 1.0),
 (2057, 2882, 1.0),
 (3327, 7898, 0.875),
 (1196, 7695, 1.0),
 (7558, 7737, 0.96875),
 (2673, 7983, 0.6717557251908397),
 (840, 1182, 0.9814814814814815),
 (884, 1656, 0.9655172413793104),
 (1612, 1708, 1.0),
 (5545, 5697, 0.975609756097561),
 (915, 1434, 0.6666666666666666),
 (9343, 9350, 0.9),
 (999, 2323, 1.0),
 (2469, 2662, 0.6666666666666666),
 (7559, 9422, 0.575),
 (2695, 3862, 0.9960474308300395),
 (903, 1465, 0.8157181571815718),
 (2973, 5966, 0.6),
 (1142, 1682, 0.8518518518518519),
 (2695, 8011, 0.5310734463276836),
 (2728, 2862, 0.75),
 (5882, 6055, 0.5882352941176471),
 (3073, 6056, 0.75),
 (499, 1837, 0.75),
 (1676, 1797, 0.8492063492063492),
 (4925, 7737, 1.0),
 (4011, 8627, 0.946236559139785),
 (2695, 9321, 0.5310734463276836),
 (261, 9089, 0.75),
 (3854, 4308, 0.8333333333333334),
 (1143